# Power Consumption Prediction 

## Overview
### The task
to estimate hourly power consumptions of today from the power consumptions history until today.

At 12 AM each day, forecasting module provides the estimated power consumptions of each hour on the day. 

Power Consumption Prediction is a task of regressions. Essentially, we can feed power consumption data in the past and features to the models to train and estimate.

Requirements are high accurate estimation. Also, computing speed may be required to provide estimation because faster computing allows to use as nearer data as possible to accurate estimation.

### Model
- Linear Regression

We used Linear Regression which is one of the most basic regression techniques.
We can also try Gaussian Process regression, Support Vector Regression, or any other regression models. Those models may give better performance however they require some tunings and computational cost. On the other hand Linear regression is very simple but it is very fast and tuning is not required. Also Linear regression can be a baseline of further improvement.

### Features
- power consumptions of previous 7 days from the target day
- hourly differences of power consumptions of previous 7 days from the target day

In terms of the data and features, we only used power consumption data to make them simple. We trained the models to estimate consumptions in a day by using previous 7 days data. Also, as extra features, we put difference between each hour and the previous hour to extract the daily trends.
This feature design allows to predict hour by hour by using like sliding windows.

### Overall test accuracy
#### Root Mean Square Error

798.7906

$\mathrm{RMSE} = \sqrt{\frac{1}{N}\sum_{n=1}^{N}(truth - estim)^2}$

#### Match Rate (rate of overlapping region between truth and estimation)

0.9586
<img src="match_rate_example.png" width="200">

### Computational speed
Both training and prediction take less than 1 second.


### Plots prediction and truth
Each plot shows the day by day estimation result (blue line), truth (red line) and input (light coloured lines). 

Surprisingly, the estimations are more accurate than we thought, especially daily trends (peaks and bottoms). However these are not perfect enough. To improve the performance, we can add some features such as weather information and seasonally trends. Also we can try more complicated models.

Estimations on Mondays, Saturdays, and Sundays seemed to be less accurate. The reason is that power consumptions on those days are different from the previous days. Modeling on each week day would solve this issue.

# import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE

from matplotlib import pyplot as plt
%matplotlib inline

import plotter

sys.path.append(os.path.join('..', 'hec'))
import utils.data_handler as dh
from utils.logger import logger
import model.preprocessing as pp
import model.model as mdl
from utils.eval import match_rate
logger.setLevel('INFO')

# Data Loading

In [ ]:
data_dir = '../data/'
n_data_to_load = 2
data = dh.data_loader(data_dir, n_data_to_load)

# data to feature vectors and target vectors
Training and testing data were divided by the ratio of 7 : 3. First 70% of the time series data was assigned for training and the rest of them was for testing.

feat_train and feat_test are the input to estimate the target values tar_train and tar_test.
As the features, we set power consumptions of 7 days before the target and the differences during the 7 days.
They have shapes of (data_length, feature_length). The feature_length is 24 hours * 7 days + 24 hours * 7 days - 1.
tar_train and tar_test have shapes of (data_length, 24 hours).

feat_train and tar_train were used for training the models. feat_test was used for estimation, and tar_test was used for checking the estimation results.

There are also some preprocessing. Power consumptions are scaled between 0 and 1. To make the structured data, if the data started from the hours which are not 12 AM, some data before 0 AM were removed.

In [ ]:
df = list(data.values())[1]
n_days = 7
norm_prm_file_name = 'norm_prm_%s.pkl' % list(data.keys())[1]
ret = pp.data_preparation(df, norm_prm_file_name, n_days)
feat_train, tar_train, feat_test, tar_test, idx_train, idx_test = ret

# Linear Regression
Linear regression is one of the most simple regression model. Before using more complicated models, we should see the estimation results of simpler models. We can obtain the estimation results to execute the cell below.

In [ ]:
l_r = mdl.Model()
l_r.fit(feat_train, tar_train)

In [ ]:
l_r_model = l_r.get_model()
print('train score:', l_r_model.score(feat_train, tar_train))
print('test score :',l_r_model.score(feat_test, tar_test))

# Prediction from trained model

In [ ]:
pred_train = l_r.predict(feat_train)
pred_test = l_r.predict(feat_test)

# Estimation results from Linear Regression

## closed evaluation (training v estimation from training) data

### data for plotting
data is put into DataFrame again for visualisations. Inverse normalisation also applied.

In [ ]:
ret = pp.post_process_array_to_df(feat_train, pred_train, tar_train, n_days, idx_train)
df_feat_nrm_trn, df_pred_nrm_trn, df_tar_nrm_trn = ret
ret = pp.post_process_inverse_norm(df_feat_nrm_trn, df_pred_nrm_trn, df_tar_nrm_trn, norm_prm_file_name)
df_feat_trn, df_pred_trn, df_tar_trn = ret

### Day by day Root Mean Square Error (RMSE)
The plot below illustrates day by day root mean square between truth and estimations. x-axis is the days and y-axis has RMSE. RMSE gives the distance between truth and prediction which means smaller values are the better.

$\mathrm{RMSE} = \sqrt{\frac{1}{N}\sum_{n=1}^{N}(truth - estim)^2}$

The RMSE at middle of the graph is higher than the other area. This may be the reason that features of this model is based on previous 7 days and on these days, temperatures might have changed significantly. For further investigation, some extra information such as weather histories may be required.

In [ ]:
plotter.plot_rmse_day_by_day(df_tar_nrm_trn, df_pred_nrm_trn)

### Overall RMSE and Match rate
Since RMSE gives the distance between truth and prediction. It is good for comparison in same scale but it is difficult to see absolute performance. So, we introduced Match rate.

Match rate gives the rate how much truth and prediction overlap each other.
```
upper = max([truth, estim])
lower = min([truth, estim])
match_rate = lower.sum() / upper.sum()
```

In [ ]:
print('RMSE(normalised):', np.sqrt(MSE(df_tar_nrm_trn.values.flatten(), df_pred_nrm_trn.values.flatten())))
print('RMSE            :', np.sqrt(MSE(df_tar_trn.values.flatten(), df_pred_trn.values.flatten())))
print('Match Rate      :', match_rate(df_tar_trn.values.flatten(), df_pred_trn.values.flatten()))

### Plots Training data and closed prediction
Each plot shows the day by day estimation result (blue line), truth (red line) and input (light coloured lines). 

Surprisingly, the estimations are more accurate than we thought, especially daily trends (peaks and bottoms). However these are not perfect enough. To improve the performance, we can add some features such as weather information and seasonally trends. Also we can try more complicated models.

Estimations on Mondays, Saturdays, and Sundays seemed to be less accurate. The reason is that power consumptions on those days are different from the previous days. Modeling on each week day would solve this issue.

In [ ]:
plotter.plot_regression_res(df_feat_trn, df_tar_trn, df_pred_trn, 7, 20)

## open evaluation (testing v estimation from testing data)

In [ ]:
ret = pp.post_process_array_to_df(feat_test, pred_test, tar_test, n_days, idx_test)
df_feat_nrm_tst, df_pred_nrm_tst, df_tar_nrm_tst = ret
ret = pp.post_process_inverse_norm(df_feat_nrm_tst, df_pred_nrm_tst, df_tar_nrm_tst, norm_prm_file_name)
df_feat_tst, df_pred_tst, df_tar_tst = ret

In [ ]:
plotter.plot_rmse_day_by_day(df_tar_nrm_tst, df_pred_nrm_tst)

In [ ]:
print('RMSE(normalised):', np.sqrt(MSE(df_tar_nrm_tst.values.flatten(), df_pred_nrm_tst.values.flatten())))
print('RMSE            :', np.sqrt(MSE(df_tar_tst.values.flatten(), df_pred_tst.values.flatten())))
print('Match Rate      :', match_rate(df_tar_tst.values.flatten(), df_pred_tst.values.flatten()))

In [ ]:
plotter.plot_regression_res(df_feat_tst, df_tar_tst, df_pred_tst, 7, 20)

# Further improvement and feature work
Mondays, Saturdays, and Sundays were not modeled well. Also, the estimations were highly related on previous days. On the other hand, as we discussed in analysis section, there are seasonally trends, weekly trends and daily trends. To model using these information, better feature design is required.

Mondays, Saturdays, and Sundays might be modeled individually, however, this approach can not use the information of previous days. So some methods to intentionally introduce trends may be required. The trends can be treated as prior information. To introduce prior information such as daily trends, we may use Bayesian inference. Bayesian approaches also allow to update the model.

Also, we haven't handled some extra cases such as extreme high power consuming, caused by weather conditions and some events like Super bowl.
To handle these cases and Saturday and Sunday problem, we may want to have some uncertainty to have decisions. Bayesian models or Gaussian processes can give such information. These approaches are worth trying if there is enough time.

Moreover, some external information like temperatures would help. To do this, more time and resources may be required.

For the use case of energy generation, shortage is worse than excess. In order to avoid this, some bias can be given in post process. Also we need to introduce some criteria to evaluate how much and often and shortage happens.

For the use case of energy trading, we can advice that we should be conservative to trade weekends and Mondays.